In [9]:
import ray
from typing import Dict, List
import numpy as np

ds = ray.data.read_json('../etl_out/test')
ds_10 = ds.limit(10)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [10]:
ray.data.DataContext.get_current().execution_options.verbose_progress = True

In [22]:

import chromadb

persistent_client = chromadb.PersistentClient('./debug_emb')
#persistent_client.delete_collection('debug')

from chromadb.utils import embedding_functions
#embedding_function = SentenceTransformerEmbeddings(model_name="/data/models/jina")
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="/data/models/bge-large-zh")
persistent_client.delete_collection("debug")
collection = persistent_client.get_or_create_collection(name="debug",metadata={"hnsw:space": "cosine"}, embedding_function=sentence_transformer_ef)


In [ ]:
for d in ds_10.iter_rows():
    #print(d['content'])
    collection.add(documents=[d['content']], ids=[d['title']])

In [26]:
collection.query(query_texts='农业')

{'ids': [['农林牧渔周报（第47期）：国内生猪行业产能去化动力增强，建议增配养猪板块',
   '农林牧渔行业周报：年末市场或不缺大猪，冻品出货压力仍较大',
   '有色金属行业周报：欧盟电网改造计划推出，工业金属需求有望提振',
   '化工行业周报：国际油价小幅下跌，部分制冷剂价格上涨',
   '新能源板块行业信息点评：欧洲北海9国将启动集体招标，计划每年拍卖15GW海风',
   '纺织服饰行业周报：冲锋衣爆火，迎品类新机遇期',
   '房地产行业研究周报：二手房同比成交改善，深圳下调二套房首付比例及放宽普宅认定标准',
   '乘用车10月月度专题：10月市场火热延续，车展催化年底车市热度不减',
   '石化行业周报：从烯烃角度看石化行业景气周期',
   '2023年中国户外运动鞋服行业研究报告：静待黎明']],
 'distances': [[0.44746041365754063,
   0.5651869836003826,
   0.5922974420484669,
   0.5946636009014914,
   0.5969260526254028,
   0.6557763763715441,
   0.6770902945376631,
   0.6802623401539305,
   0.6940730402160646,
   0.7155100718466154]],
 'metadatas': [[None, None, None, None, None, None, None, None, None, None]],
 'embeddings': None,
 'documents': [["\\u3000\\u3000一、市场回顾 \\n', '\n\\u3000\\u3000农业指数大幅上涨，板块表现明显优于大市。 1、 上周，申万农业指数上涨 3.27%，同期，上证综指下跌 0.43%，深成指下跌 1.4%，农业板块整体表现明显优于大市。 2、 细分来看， 6 个子行业指数全部上涨，其中渔业和养殖涨幅领先。 3、 前 10 大涨幅个股主要集中在养殖板块，前 10 大跌幅个股集中在动保板块。涨幅前 3 名个股依次为国联水产、神农集团和金新农。\\n', '\n\\u3000\\u3000二、 核心观点\

In [14]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoModel
from llama_index.core.node_parser import  SentenceSplitter #SimpleNodeParser
from llama_index.core.schema import Node
from llama_index.core.schema import Document
import pandas as pd
import pyarrow as pa
import warnings
import pandas as pd
import copy

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

def convert_documents_into_nodes(data: Dict[str, np.ndarray]) -> List[Dict[str, Node]]:
    splitter = SentenceSplitter()
    all_text = data['title'] + data['content']
    doc = Document(text = all_text )
    del data["title"]
    del data["content"]
    doc.metadata = data
    documents = [ doc]
    nodes = splitter.get_nodes_from_documents(documents=documents)
    return [{"node": node} for node in nodes]

class EmbedNodes:
    def __init__(self):
        import os
        #os.environ['HTTPS_PROXY'] = 'http://100.109.83.118:808/'
        self.embedding_model = HuggingFaceEmbeddings(model_name='/data/models/jina')

    #列存储的方式
    def __call__(self, node_batch: Dict[str, List[Node]]) -> Dict[str, List[Node]]:
       
        nodes = node_batch["node"]
        text = [node.text for node in nodes]
        embeddings = self.embedding_model.embed_documents(text)
        assert len(nodes) == len(embeddings)

        for node, embedding in zip(nodes, embeddings):
            node.embedding = embedding
        return {"embedded_nodes": nodes}

In [15]:
embeds_10 = ds_10.flat_map(convert_documents_into_nodes).map_batches(EmbedNodes, concurrency=4, num_gpus=1, batch_size=100)
tmp_nodes = []
for row in embeds_10.iter_rows():
    node = row["embedded_nodes"]
    assert node.embedding is not None
    tmp_nodes.append(node)

2024-03-10 12:46:09,887	INFO set_read_parallelism.py:115 -- Using autodetected parallelism=96 for stage ReadJSON to satisfy parallelism at least twice the available number of CPUs (48).
2024-03-10 12:46:09,888	INFO set_read_parallelism.py:122 -- To satisfy the requested parallelism of 96, each read task output is split into 96 smaller blocks.
2024-03-10 12:46:09,891	INFO streaming_executor.py:112 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadJSON] -> LimitOperator[limit=10] -> TaskPoolMapOperator[FlatMap(convert_documents_into_nodes)] -> ActorPoolMapOperator[MapBatches(EmbedNodes)]
2024-03-10 12:46:09,893	INFO streaming_executor.py:113 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), exclude_resources=ExecutionResources(cpu=0, gpu=0, object_store_memory=0), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-03-10 12:46:09,895	INFO streamin

2024-03-10 12:46:09,984	INFO actor_pool_map_operator.py:114 -- MapBatches(EmbedNodes): Waiting for 4 pool actors to start...


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

2024-03-10 12:46:24,875	WARNING actor_pool_map_operator.py:278 -- To ensure full parallelization across an actor pool of size 4, the Dataset should consist of at least 4 distinct blocks. Consider increasing the parallelism when creating the Dataset.


In [16]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = Ollama(model="qwen:7b", request_timeout=120.0)
Settings.embed_model = HuggingFaceEmbedding(
    model_name="/data/models/jina"
)

Some weights of BertModel were not initialized from the model checkpoint at /data/models/jina and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermediate.dense.weight

In [29]:
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext


# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("debug_1")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create your index
index = VectorStoreIndex(
    nodes=tmp_nodes, storage_context=storage_context
)

In [1]:
# import
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document
model_name = "/data/models/bge-large-zh"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

embedding_function = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)


docs = [Document(page_content='我爱北京和上海')]
db = Chroma.

In [6]:
query = "我爱北京和上海"
docs = db.similarity_search_with_relevance_scores(query)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


In [7]:
docs

[(Document(page_content='我爱北京和上海'), 0.8808702230453491)]

In [8]:
import chromadb

client = chromadb.PersistentClient(path="./chroma_db")
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from chromadb.utils import embedding_functions
#embedding_function = SentenceTransformerEmbeddings(model_name="/data/models/jina")
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="/data/models/bge-large-zh")
client.delete_collection("debug_1")
collection = client.get_or_create_collection(name="debug_1", embedding_function=sentence_transformer_ef)

Some weights of BertModel were not initialized from the model checkpoint at /data/models/jina and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermediate.dense.weight

In [22]:
collection.add(documents=["I love car"], ids=['b'])


In [23]:
collection.peek()

{'ids': ['a', 'b'],
 'embeddings': [[-1.931208848953247,
   -1.4851881265640259,
   -0.4203472137451172,
   -1.9734103679656982,
   0.7658465504646301,
   -0.1711435317993164,
   -1.196637511253357,
   -0.655268669128418,
   1.5780582427978516,
   0.7941847443580627,
   -1.3588402271270752,
   -0.18895389139652252,
   -0.8426016569137573,
   0.5383318066596985,
   0.09754922240972519,
   -0.12517070770263672,
   -0.40114840865135193,
   -0.7396183013916016,
   1.072955846786499,
   -0.35865622758865356,
   -0.2281494140625,
   -0.3829073905944824,
   -0.13564303517341614,
   2.8097891807556152,
   0.47759753465652466,
   0.6095921993255615,
   3.3343772888183594,
   0.7554832696914673,
   0.2291887253522873,
   -2.0141983032226562,
   2.4645631313323975,
   1.085907220840454,
   -0.2715925872325897,
   0.5408307909965515,
   -0.6899696588516235,
   -0.40769705176353455,
   -0.20089633762836456,
   -1.6518827676773071,
   1.924743890762329,
   -0.7904072999954224,
   -0.7380453944206238

In [27]:
collection.query(
    query_texts=["I like car"],
    n_results=1,
    include=['distances']
)

{'ids': [['a']],
 'distances': [[395.09427100260143]],
 'metadatas': None,
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None}